I am revisiting the second assignment, as I didn't fully grasp its goals during my initial attempt, which resulted in an underwhelming outcome.

As i am doing this nootebook after class, i allready now that the ranker is basically a "popularity ranking", as such, i will try and improve said metric.

I would like to acknowledge that I based this notebook on the work of Jakub Niedziela, as he was the one to come up with the solution

note: my initial work is in the same folder, named equally to this one but adding "_old"

In [1]:
from lightgbm.sklearn import LGBMRanker
import numpy as np
import pandas as pd

We import the cleaned csv files.  
They are the same csv data but with some key differences:

    Label encoded (takes less memory space)
    Normalized
    Added weeks to transactions (important!)

In [2]:
transactions = pd.read_parquet('../00 - Data/transactions_train/transactions_train.parquet')
customers = pd.read_parquet('../00 - Data/customers/customers.parquet')
articles = pd.read_parquet('../00 - Data/articles/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

In [3]:
#get last week +1 (we are trying to predict next weeks data)
test_week = transactions.week.max() + 1
#we filter data from last 10 weeks, as anything previous (summer clothes for example) might not be entirely necessary
transactions = transactions[transactions.week > transactions.week.max() - 10]

Generating candidates


In [4]:
#we get a list of every customer who has bought in each week
c2weeks = transactions.groupby('customer_id')['week'].unique()

#this shows the limits for each week. It works fine
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [5]:
#shifts weeks one week, so we've got an aditional one
c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

#replace shifted weeks

candidates_last_purchase = transactions.copy()

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

Bestsellers candidates

In [6]:
#mean price of each article for each week
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

#"dense" will rank equally two items with same values (number of times sold in a week) 1-2-2-3 
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

#for each week it gives the top 12 most selled items
sales

# join sales rank wirh average price on each article - week
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1
#we include the price of each of the best selled items
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [7]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

#just gets the list of customers who made a transaction (duplicates are possible) for each of the weeks
unique_transactions

#leaves a list of customers per week
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [8]:
#join on customer + best articles purchased (rank and price)
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)
candidates_bestsellers

#get only unique customers
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week


#join on test_set customers + best articles purchased (rank and price)
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

#data + test data
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

Combining transactions and candidates / negative examples

In [9]:
# As all data here is historical, we know for fact that all purchases actually occured
assert transactions.week.max() != test_week, 'Test week should not be in historical data'

#we will later add some sample data that will have this column set to 0
transactions['purchased'] = 1

#fill dataframe with negative values, everything that was not bought is a sample now
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

#transactions = customer - week grouping with article bought and price
display(transactions.sample(3))
#shifted weeks in transactions
display(candidates_last_purchase.sample(3))
#join on customer + best articles purchased price
display(candidates_bestsellers.sample(3))

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
31258500,2020-09-08,6162030673331763443,748355003,0.022017,2,102,1
31584739,2020-09-17,7779277796119032824,895555001,0.042356,2,104,1
29159065,2020-07-17,18144024652367926358,739265003,0.016932,2,95,1


,t_dat,customer_id,article_id,price,sales_channel_id,week
29903365,2020-08-04,12588518229110888803,832473003,0.016932,1,103
29231512,2020-07-20,1844888445343397008,811368001,0.015237,2,96
29846279,2020-08-03,11140663282535151277,823505001,0.030492,2,105


,t_dat,customer_id,sales_channel_id,week,article_id,price
7909746,2020-09-20,12707829296586431976,2,104,809238001,0.041612
3167323,2020-08-13,12420156350657910531,2,99,739144004,0.010689
2552663,2020-08-08,16155737450858897096,2,98,827968001,0.016618


Feature Engineering

What kind of things can we do to improve the model? 

In [10]:
#Most Sold in Last 10 weeks

# Increased the score
# 0.02046 -> 0.02049
bestsellers_last_n_weeks = transactions['article_id'].value_counts()\
    .reset_index()
bestsellers_last_n_weeks['bestseller_rank_n_weeks'] = bestsellers_last_n_weeks['count'].rank(method='dense', ascending=False)
bestsellers_last_n_weeks.drop(columns='count', inplace=True)

In [11]:
#bestsellers_last_n_weeks

# Increased the score
# 0.02049 -> 0.02051
customer_favourite_color = \
    pd.merge(transactions[['customer_id', 'article_id']], articles[['article_id', 'perceived_colour_master_id']], on='article_id')\
    .drop(columns='article_id')\
    .groupby('customer_id')['perceived_colour_master_id'].value_counts()\
    .groupby('customer_id').head(1)\
    .reset_index()\
    .drop(columns='count')\
    .rename(columns={'perceived_colour_master_id': 'fav_perceived_colour_master_id'})   

In [12]:
# Bestsellers by postal code

# Lower score
# 0.02051 -> 0.02042
bestsellers_zipcode = \
    pd.merge(transactions[['customer_id', 'article_id']], customers[['customer_id', 'postal_code']], on='customer_id') \
    .drop(columns='customer_id') \
    .groupby('article_id')['postal_code'].value_counts() \
    .groupby('article_id').head(1) \
    .reset_index() \
    .drop(columns='count') \
    .rename(columns={'postal_code': 'most_popular_postal_code'})

In [13]:
# Customers favourite channel

customer_favourite_channel = \
    pd.merge(transactions[['customer_id', 'article_id', 'sales_channel_id']], articles[['article_id']], on='article_id')\
    .drop(columns='article_id')\
    .groupby('customer_id')['sales_channel_id'].value_counts()\
    .groupby('customer_id').head(1)\
    .reset_index()\
    .drop(columns='count')\
    .rename(columns={'sales_channel_id': 'fav_sales_channel_id'})

In [14]:
# min, avg and max product price

# Increased the score
# 0.02042 -> 0.02094
article_prices = transactions[['article_id', 'price']].groupby('article_id').agg(['min', 'max', 'mean']).reset_index()
article_prices.columns = ['article_id', 'min_article_price', 'max_article_price', 'mean_article_price']

In [15]:
# min, avg, max purchase price

customer_prices = transactions[['customer_id', 'price']].groupby('customer_id').agg(['min', 'max', 'mean']).reset_index()
customer_prices.columns = ['customer_id', 'min_customer_price', 'max_customer_price', 'mean_customer_price']

COMBINE DATA

In [16]:
model_data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

# Remove first week of data, as we don't have bestseller rank for it
# (week was shifted by one) and fill missing values with 999 -- really bad rank
model_data = model_data[model_data.week != model_data.week.min()]
model_data.fillna({'bestseller_rank':999}, inplace=True)

# Add customer and article features
model_data = pd.merge(model_data, articles, on='article_id', how='left')
model_data = pd.merge(model_data, customers, on='customer_id', how='left')

# Add bestseller rank for last n weeks
model_data = pd.merge(model_data, bestsellers_last_n_weeks, on='article_id', how='left')

# Add customer favourite color
model_data = pd.merge(model_data, customer_favourite_color, on='customer_id', how='left')

# Add customer favourite channel
model_data = pd.merge(model_data, customer_favourite_channel, on='customer_id', how='left')

# Add most popular postal code for each article
model_data = pd.merge(model_data, bestsellers_zipcode, on='article_id', how='left')

# Add article price features
model_data = pd.merge(model_data, article_prices, on='article_id', how='left')

# Add customer price features
model_data = pd.merge(model_data, customer_prices, on='customer_id', how='left')

In [17]:
#reorganizes the model_data dataframe bsaed on week and customer_id
model_data.sort_values(['week', 'customer_id'], inplace=True)
#drops current index (maybe 123 - 1 - 943 - 40 - ..) and creates a new one (1 - 2 - 3 - 4)
model_data.reset_index(drop=True, inplace=True)

In [18]:
#we want all data but our test one to train
train = model_data[model_data.week != test_week]

test = model_data[model_data.week == test_week]\
    .drop_duplicates(['customer_id', 'article_id', 'sales_channel_id'])\
    .copy()

In [19]:
# Basically how many purchased for each customer week pair -- so lgbm knows its one transaction
train_baskets = train.groupby(['week', 'customer_id'])['article_id']\
    .count()\
    .values  
# for lightgbm ranker fit method
# groupnumpy 1-D array
#     Group/query data. Only used in the learning-to-rank task. sum(group) = n_samples. 
#     For example, if you have a 100-document dataset with group = [10, 20, 40, 10, 10, 10], that means that you have 6 groups, 
#     where the first 10 records are in the first group, records 11-30 are in the second group, records 31-70 are i

In [22]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank', 
    'bestseller_rank_n_weeks', 
    'fav_perceived_colour_master_id',
    # 'fav_sales_channel_id',
    # 'most_popular_postal_code',
    # 'min_article_price', 'max_article_price', 'mean_article_price',
    # 'min_customer_price', 'max_customer_price', 'mean_customer_price',
    ]

In [23]:
train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

MODEL TRAINING

In [24]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [25]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.848530
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.192180
[LightGBM] [Debug] init for col-wise cost 0.193828 seconds, init for row-wise cost 0.758255 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.519700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1343
[LightGBM] [Info] Number of data points in the train set: 11849508, number of used features: 20
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
CPU times: total: 26.7 s
Wall time: 10.3 s


In [26]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9975670947443732
fav_perceived_colour_master_id 0.001178745275527028
colour_group_code 0.00032532053061590075
perceived_colour_master_id 0.0002543311364503016
article_id 0.00024399601970804725
age 0.00016456009626360316
bestseller_rank_n_weeks 0.00010073233227161269
garment_group_no 6.230984900730054e-05
postal_code 3.9511089501591246e-05
perceived_colour_value_id 3.5539264754624535e-05
product_type_no 2.7859661526759114e-05
fashion_news_frequency 0.0
club_member_status 0.0
Active 0.0
FN 0.0
index_group_no 0.0
index_code 0.0
department_no 0.0
graphical_appearance_no 0.0
section_no 0.0


CALCULATE PREDICTIONS

In [27]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

# Get bestsellers from last week (test week)
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: total: 0 ns
Wall time: 0 ns


CREATE SUBMISSION

In [28]:
preds = []
for c_id in customers.customer_id:
    # c_id = idx_to_cust_id[c_id]  # map to original customer id
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

In [ ]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub = customers[['customer_id']].copy()

In [ ]:
# This part is due to reduction mapping for customer ids

with open('../data/mappings/cust_id_to_idx.json', 'r') as f:
    cust_id_to_idx = json.load(f)

idx_to_cust_id = {v: k for k, v in cust_id_to_idx.items()}

sub['customer_id'] = sub['customer_id'].apply(lambda x: idx_to_cust_id[x])

In [ ]:
sub['prediction'] = preds

In [ ]:
sub_name = 'basic_model_submission__best_features'
sub.to_csv('./data/submissions/{sub_name}.csv.gz', index=False)
!kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f '../data/submissions/{sub_name}.c

In [ ]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 3